### TME sur Echantillonage

## Diffusion dans les graphes 

Au cours des vingt dernières années, les réseaux sociaux sont devenus un média d’information incontournable, mettant en jeu des dynamiques complexes de communication entre utilisateurs. La modélisation de la diffusion d’information sur les réseaux constitue depuis lors un enjeu majeur, pour diverses tâches
telles que l’identification de leaders d’opinions, la prédiction ou la maximisation de l’impact d’un contenu diffusé, la détection de communautés d’opinions, ou plus généralement l’analyse des dynamiques du réseau considéré.

Le modèle proposé par (Saito et al, 2009) considère une diffusion en cascade dans laquelle l'information transite de noeuds en noeuds du réseau en suivant des relations d'influence entre les utilisateurs. Lorsqu'un utilisateur est ``infecté'' par une information, il possède une chance unique de la retransmettre à chacun de ses successeurs dans le graphe, selon une probabilité définie sur le lien correspondant. Le modèle définit en fait deux paramètres sur chaque lien $(u,v)$ du graphe:


*   $k_{u,v}$: la probabilité que l'utilisateur $u$ transmette une information diffusée à $v$
*   $r_{u,v}$: si la transmission s'effectue, l'utilisateur $v$ la reçoit au temps $t_v=t_u+\delta$, avec $\delta \sim Exp(r_{u,v})$

Pour utiliser ce modèle, on devra donc échantillonner selon la distribution exponentielle. Pour commencer, on cherche alors à écrire une méthode $exp(rate)$ qui échantillonne des variables d'une loi exponentielle selon le tableau d'intensités $rate$ passé en paramètre. Cet échantillonnage se fera par **Inverse Transform Sampling**. Pour éviter les divisions par 0, on ajoutera $1e-200$ aux intensités qui valent 0.  


In [1]:
import numpy as np
np.random.seed(0)
def exp(rate): # note: rate correspond à lambda dans les formules usuelles de la loi exponentielle
    #>>>>>>>>>>
    # votre code ici
    # 1. Avoir calculé x = F(u, rate) comme en TD
    # 2. Retourner un tirage aléatoire à partir de np.random.rand
    # note: si on donne plusieurs valeur de rate, on fera autant de tirages (comme ci-dessous)
    #<<<<<<<<<<
    rate = np.where(rate==0,1e-200,rate)
    x = np.random.rand(*rate.shape)
    return -np.log(1.0-x)/rate

#Test : on sait que l'espérance de la loi exp est 1/lambda 
a=exp(np.array([[1,2,3],[4,5,6]]))
for i in range(10000):
    a+=exp(np.array([[1,2,3],[4,5,6]]))
print(a/10000) # calcul de l'espérance

# Pour comparaison avec la méthode de référence de numpy:
# ATTENTION, la méthode attend un paramètre 1/lambda (et non lambda)
a=np.random.exponential(1.0/np.array([[1,2,3],[4,5,6]]))
for i in range(10000):
    a+=np.random.exponential(1.0/np.array([[1,2,3],[4,5,6]]))
print(a/10000) # calcul de l'espérance


[[0.98796784 0.49198855 0.33501196]
 [0.25022762 0.19644862 0.16723749]]
[[1.00356177 0.50416273 0.34028414]
 [0.25231623 0.20024732 0.16911951]]


Vérification :



```
[[0.98796784 0.49198855 0.33501196]
 [0.25022762 0.19644862 0.16723749]]
[[1.00356177 0.50416273 0.34028414]
 [0.25231623 0.20024732 0.16911951]]
```



Soit le graphe de diffusion donné ci dessous: 

In [2]:

names={0:"Paul",1:"Jean",2:"Hector",3:"Rose",4:"Yasmine",5:"Léo",6:"Amine",7:"Mia",8:"Quentin",9:"Gaston",10:"Louise"}
k={(0,1):0.9,(1,0):0.9,(1,2):0.2,(2,3):0.5,(3,2):0.4,(2,4):0.9,(4,3):0.9,(1,3):0.5,(2,5):0.5,(5,7):0.7,(1,6):0.2,(6,7):0.1,(1,8):0.8,(8,9):0.2,(1,10):0.5,(10,9):0.9,(8,1):0.8}
r={(0,1):0.2,(1,0):3,(1,2):1,(2,3):0.2,(3,2):0.5,(2,4):10,(4,3):2,(1,3):2,(2,5):0.5,(5,7):15,(1,6):3,(6,7):4,(1,8):0.8,(8,9):0.1,(1,10):12,(10,9):1,(8,1):14}
graph=(names,k,r)


La fonction display_graph ci dessous permet de visualiser le graphe de diffusion correspondant: 

In [3]:
import pydot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

style = { "bgcolor" : "#6b85d1", "fgcolor" : "#FFFFFF" }

def display_graph ( graph_data, style, graph_name="diffusion_graph" ):
    graph = pydot.Dot( graph_name , graph_type='digraph')
    names,k,r=graph_data
    # création des noeuds du réseau
    for (i,name) in names.items():
        new_node = pydot.Node( str(i)+"_"+name,
                               style="filled",
                               fillcolor=style["bgcolor"],
                               fontcolor=style["fgcolor"] )
        graph.add_node( new_node )

    # création des arcs
    for edge,valk in k.items():
        valr=r[edge]
        n1=str(edge[0])+"_"+names[edge[0]]
        n2=str(edge[1])+"_"+names[edge[1]]
        new_edge = pydot.Edge ( n1, n2, label="k="+str(valk)+",r="+str(valr))
        graph.add_edge ( new_edge )

    # sauvegarde et affichage
    outfile = graph_name + '.png'
    graph.write_png( outfile )
    img = mpimg.imread ( outfile )
    plt.imshow( img )
display_graph(graph,style)

FileNotFoundError: [Errno 2] "dot" not found in path.

On souhaite être capable d'estimer les probabilités marginales d'infection des différents utilisateurs du réseau par une information pour laquelle on connaît les sources (i.e., les utilisateurs infectés au temps 0). 

Etant donnés les cycles possibles dans le graphe de diffusion, considérer un calcul exact des probabilités d'infection des différents utilisateurs sachant le début de la diffusion est inenvisageable : il faudrait considérer toutes les combinaisons possibles (infinies) de temps d'infection pour tous les utilisateurs non sources. 

Une possibilité pour calculer ces probabilités d'infections est de travailler par échantillonnage de Monte Carlo: on réalise $n$ tirages d'infections connaissant les sources et on recense le ratio des simulations dans lesquelles chacun des utilisateurs est infecté avant un temps $maxT$.  

L'idée est alors dans un premier temps d'écrire une méthode $simulation(graph,sources)$ qui, à partir d'une liste de sources, retourne les temps d'infection de l'ensemble des noeuds en fin de diffusion, sous la forme d'un tableau où chaque case $i$ contient le temps d'infection du noeud $i$. Si le noeud $i$ n'a pas été infecté ou bien si il l'a été après un temps maximal $maxT$, la case $i$ contient alors la valeur $maxT$. 

Le pseudo-code de la méthode de simulation est donné ci dessous, avec $t_i$ le temps d'infection courant du noeud $i$:
```
ti=maxT pour tout i non source 
Tant qu'il reste des infectieux dont le temps est < maxT:
  i=Infectieux de temps d'infection minimal
  Pour tout noeud j tel que tj>ti:
    sampler x selon Bernoulli(kij)
    si x==1:
       sampler delta selon Exp(rij)
       t=ti+delta  
       si t<tj: tj=t 
  Retrait de i de la liste des infectieux
```
Complétez le code de la fonction donnée ci-dessous: 

**Note:** les résultats de référence ne seront obtenus que si on fait les appels à random dans le même ordre que dans dans la correction de référence... Ce sera le cas si vous suivez les consignes détaillées ci-dessous. Mais vous pouvez aussi tenter de travailler directement à partir de l'algorithme ci-dessus.

In [4]:
np.random.seed(0)
maxT=10

# returns dense numpy arrays of k,r parameters for graph links fr -> to 
def get_kr_for(graph,fr,to):
    _,gk,gr=graph
    k = np.array([[gk.get((i, v),0) for v in to] for i in fr])
    r = np.array([[gr.get((i, v),0) for v in to] for i in fr])
    return k,r

def simulation(graph,sources, maxT):
    nbNodes=len(names)
    infectious = np.ones(len(graph[0]))*maxT
    for source in sources:
        infectious[source] = 0
    times = np.copy(infectious)
    
    # On boucle tant qu'il reste des contaminants (ie: qu'il reste des 0 dans infectious)
    while 0 in infectious:
        # trouver le noeud contaminant à cette itération = argmin dans infectious
        Noeud_contaminant = np.argmin(infectious)
        
        # trouver le temps associé à la contamination: Tref
        Tref = times[Noeud_contaminant]
        
        # éliminer le noeud en mettant sa valeur à maxT dans infectious => il ne sera plus sélectionné
        infectious[Noeud_contaminant] = maxT
        
        # trouver les indices des cibles (temps de contamination > Tref)
        targets = np.where(times > Tref)[0]

        # trouver les paramètres des modèles entre le noeud source et les cibles:     
        params = get_kr_for(graph,[Noeud_contaminant], targets) # récupération des paramètres vers les cibles

        for j in range(len(targets)):
            # tirage Bernoulli selon params[0][0]: les cibles sont elles contaminées
            # La noeud infectieux infecte la cible j avec une probabilité p stockée dans params (parsée dans graph)
            # On fait un test de Bernouilli (ie: binomiale avec un seul tirage)
            p = params[0][0][j]
            if np.random.binomial(1, p, 1):
                # Si la cible est infectée on fait un tirage exponentiel pour connaître son temps de contamination
                # (Tref + tirage) selon son lambda dans params[1][0]
                lam = params[1][0][j]
                ti = Tref + exp(lam)
                if ti < times[targets[j]]:
                    times[targets[j]] = ti
                    infectious[targets[j]] = 0
    return times

np.random.seed(1)
print(simulation(graph,[0], maxT))
print(simulation(graph,[0], maxT))
print(simulation(graph,[0], maxT))
np.random.seed(1)
print(simulation(graph,[0,1], maxT))
print(simulation(graph,[0,1], maxT))
print(simulation(graph,[0,1], maxT))


[ 0.          6.37062627 10.         10.         10.         10.
 10.         10.          6.55971741  8.45768892  6.6581452 ]
[ 0.          2.97007182 10.          3.04941061 10.         10.
 10.         10.          3.63849864  4.03296435  3.06090791]
[ 0. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
[0.00000000e+00 0.00000000e+00 1.00000000e+01 1.00000000e+01
 1.00000000e+01 1.00000000e+01 9.25654159e-03 1.00000000e+01
 1.02250914e+00 1.00000000e+01 1.00000000e+01]
[ 0.          0.         10.         10.         10.         10.
 10.         10.          1.41975241 10.         10.        ]
[ 0.          0.         10.         10.         10.         10.
 10.         10.          1.73693508 10.          0.08144123]


Vérification : 

```
[ 0.          2.71669685 10.         10.         10.         10.
 10.         10.          3.19055869  3.17528764  2.86665883]
[ 0.          0.60940319 10.         10.         10.         10.
 10.         10.          2.36988928 10.         10.        ]
[ 0.          0.22787406 10.         10.         10.         10.
 10.         10.          1.27950225  3.42920125 10.        ]
[ 0.          0.          0.03983788  0.09306264  0.05063365  1.10889995
 10.          1.16647819 10.          1.16739272  0.03159079]
[ 0.          0.         10.         10.         10.         10.
  0.16359844 10.          1.71855838 10.         10.        ]
[ 0.          0.          3.08047501  1.49963044  3.25699405 10.
 10.         10.          0.83189232  2.23597755 10.        ]
```



La méthode $getProbaMC(graph,sources,nbsimu)$ retourne les estimations de probabilités marginales d'infection des différents noeuds de $graph$, conditionnées à l'observation des  $sources$. Pour être enregistrée, une infection doit intervenir avant la seconde $maxT$. Ainsi, si la méthode retourne 0.2 pour le noeud $i$, cela indique qu'il a été infecté avec un temps $t_i \in ]0,maxT[$ dans 20% des $nbsimu$ simulations effectuées. Compléter la méthode ci dessous: 

In [5]:
np.random.seed(0)

def getProbaMC(graph,sources, maxT, nbsimu=100000):
    names,gk,gr=graph # eclatement du graphe
    nbNodes=len(names)
    rInf= np.zeros(nbNodes) # nb d'infection de chaque noeud dans la simulation suivante
    # boucle for sur nbsimu
    for k in range(nbsimu):
        #Réalisation d'une simulation
        sim = simulation(graph,sources,maxT)
        #Incrément pour les noeuds contaminés dans la simulation
        rInf += np.where((sim>0) & (sim<maxT),1,0)    #d'après la formule de l'énoncé
    #retour de rInf (normalisé en fréquence et pas en comptage)
    return rInf / nbsimu
    

#On obtient des 0 à la place des 1 car l'énoncé nous dit de ne pas prendre en compte les infections au temps 0 (donc ceux des sources)
#Alors que dans la vérification il semblerait que l'on prenne en compte les temps des sources
rInf=getProbaMC(graph,[0], maxT)
print(rInf) 

rInf=getProbaMC(graph,[0], maxT)
print(rInf)

rInf=getProbaMC(graph,[0,1], maxT)
print(rInf)  

rInf=getProbaMC(graph,[2,8], maxT)
print(rInf) 


[0.      0.77642 0.25572 0.44175 0.22981 0.11008 0.15189 0.08933 0.58858
 0.36184 0.38566]
[0.      0.77608 0.25869 0.44728 0.2316  0.1128  0.15182 0.09141 0.58873
 0.36193 0.38589]
[0.      0.      0.35815 0.58964 0.32316 0.17678 0.20182 0.141   0.80083
 0.50104 0.49992]
[0.72278 0.80247 0.      0.93594 0.89964 0.49761 0.1597  0.35829 0.
 0.44057 0.39873]


Vérification : 


```
[1.      0.7785  0.25939 0.44694 0.23214 0.11123 0.15518 0.09145 0.58973
 0.36455 0.38976]
[1.      0.77994 0.25928 0.44709 0.23307 0.11118 0.155   0.09067 0.59052
 0.36201 0.38788]
[1.      1.      0.35724 0.58993 0.32084 0.17582 0.20088 0.13995 0.79891
 0.49967 0.49876]
[0.71818 0.79804 1.      0.93559 0.89997 0.49813 0.15957 0.35803 1.
 0.44108 0.39904]
```



Cette méthode permet de bonnes estimations (malgré une certaine variance) lorsque l'on n'a pas d'observations autres que le vecteur de sources (i.e., on estime des probabilités de la forme: $P(t_i < maxT|\{(j,t_j),t_j=0\})$). Par contre, si l'on souhaite obtenir des probabilités d'infection du type $P(t_i < maxT|\{(j,t_j),t_j=0\}, \{(j,t_j), j \in {\cal O}\})$, c'est à dire conditionnées à des observations supplémentaires pour un sous-ensembles de noeuds ${\cal O}$ (avec $t_j > 0$ pour tout noeud $j$ de ${\cal O}$), l'utilisation de la méthode de MonteCarlo précédente est impossible. Cela impliquerait de filtrer les simulations obtenues selon qu'elles remplissent les conditions sur les noeuds de ${\cal O}$, ce qui nous amènerait à toutes les écarter sachant que l'on travaille avec des temps continus. 

Pour estimer ce genre de probabilité conditionnelle, nous allons nous appuyer sur des méthodes de type MCMC, notamment la méthode de Gibbs Sampling. Cette méthode est utile pour simuler selon une loi jointe, lorsqu'il est plus simple d'échantillonner de chaque variable conditionnellement à toutes les autres plutôt que directement de cette loi jointe. L'algorithme est donné par: 


1.   Tirage d'un vecteur de valeurs initiales pour toutes les variables $X_i$
2.   Pour toutes les variable $X_i$ choisies dans un ordre aléatoire, échantillonnage d'une nouvelle valeur: $X_i \sim p(x_i\mid x_1,\dots,x_{i-1},x_{i+1},\dots,x_n)$
3.   Recommencer en 2 tant qu'on souhaite encore des échantillons

Notons qu'il est souvent utile d'exploiter la relation suivante, qui indique que pour échantillonner de la loi conditionnelle, il suffit d'échantillonner chaque variable proportionnellement à la loi jointe, avec toutes les autres variables fixées: 
$$p(x_j\mid x_1,\dots,x_{j-1},x_{j+1},\dots,x_n) = \frac{p(x_1,\dots,x_n)}{p(x_1,\dots,x_{j-1},x_{j+1},\dots,x_n)} \propto p(x_1,\dots,x_n)$$

Après une période dite de $burnin$ d'un nombre d'époques à définir, l'algorithme émet des échantillons qui suivent la loi jointe connaissant les observations. Lorsque l'objectif est d'estimer des probabilités marginales, on fait alors tourner cet algorithme pendant une certain nombre d'époques après la période de $burnin$, au cours desquelles on recence les différentes affectations de chacune des variables étudiées. 

Pour mettre en oeuvre cet algorithme, nous aurons aurons besoin d'avoir accès rapidement aux prédecesseurs et successeurs dans le graphe. La méthode ci-dessous retourne un couple de dictionnaires à partir du graphe: 
 

*   $preds[i]$  contient la liste des prédécesseurs du  noeud $i$, sous la forme d'une liste de triplets $(j,k_{j,i},r_{j,i})$ pour tous les $j$ précédant $i$ dans le graphe.    
*   $succs[i]$  contient la liste des successeurs du  noeud $i$, sous la forme d'une liste de triplets $(j,k_{i,j},r_{i,j})$ pour tous les $j$ pointés par $i$ dans le graphe.





In [6]:
# pré-calcul des précécesseurs et successeurs pour gagner du temps ensuite
def getPredsSuccs(graph):
    names,gk,gr=graph
    nbNodes=len(names)
    preds={}
    succs={}
    for (a,b),v in gk.items():
        s=succs.get(a,[])
        s.append((b,v,gr[(a,b)]))
        succs[a]=s
        p=preds.get(b,[])
        p.append((a,v,gr[(a,b)]))
        preds[b]=p
    
    return (preds,succs)

preds,succs=getPredsSuccs(graph)
print("preds=",preds)
print("succs=",succs)

preds= {1: [(0, 0.9, 0.2), (8, 0.8, 14)], 0: [(1, 0.9, 3)], 2: [(1, 0.2, 1), (3, 0.4, 0.5)], 3: [(2, 0.5, 0.2), (4, 0.9, 2), (1, 0.5, 2)], 4: [(2, 0.9, 10)], 5: [(2, 0.5, 0.5)], 7: [(5, 0.7, 15), (6, 0.1, 4)], 6: [(1, 0.2, 3)], 8: [(1, 0.8, 0.8)], 9: [(8, 0.2, 0.1), (10, 0.9, 1)], 10: [(1, 0.5, 12)]}
succs= {0: [(1, 0.9, 0.2)], 1: [(0, 0.9, 3), (2, 0.2, 1), (3, 0.5, 2), (6, 0.2, 3), (8, 0.8, 0.8), (10, 0.5, 12)], 2: [(3, 0.5, 0.2), (4, 0.9, 10), (5, 0.5, 0.5)], 3: [(2, 0.4, 0.5)], 4: [(3, 0.9, 2)], 5: [(7, 0.7, 15)], 6: [(7, 0.1, 4)], 8: [(9, 0.2, 0.1), (1, 0.8, 14)], 10: [(9, 0.9, 1)]}


Pour calculer les probabilités conditionnelles, il faut prendre en compte les quantités suivantes: 


*   Probabilité pour $j$ d'être infecté par $i$ au temps $t_j$ connaissant $t_i < t_j$:  
$$\alpha_{i,j}=k_{i,j}r_{i,j} exp(-r_{i,j}(t_j-t_i))$$
*   Probabilité pour $j$ de ne pas être infecté par $i$ jusqu'au temps $t$:
$$\beta_{i,j}=k_{i,j} exp(-r_{i,j}(t_j-t_i)) + 1 - k_{i,j}$$
*   Probabilité pour $j$ d'être infecté au temps $t_j$ connaissant les prédecesseurs infectés avant $t_j$:
$$h_{j}=\prod_{i \in preds[j], t_i<t_j} \beta_{i,j} \sum_{i \in preds[i], t_i<t_j} \alpha_{i,j} / \beta_{i,j}$$
*   Probabilité pour $j$ de ne pas être infecté avant $maxT$ connsaissant ses prédecesseurs infectés:
$$g_{j}=\prod_{i \in preds[j], t_i<t_j} \left(k_{i,j} exp(-r_{i,j}(maxT-t_i)) + 1 - k_{i,j}\right)=\prod_{i \in preds[j], t_i<t_j} \beta_{i,j}$$




Dans la méthode $computeab(v, times, preds)$, on prépare le calcul et les mises à jour de ces quantités. La méthode calcule, pour un noeud $v$ selon les temps d'infection courants donnés dans $times$, deux quantités $a$ et $b$: 

$$a= \left\{
\begin{array}{l}
\max(1e^{-20}, \sum_{i \in preds[v], t_i<t_v} \alpha_{i,v} / \beta_{i,v}) \mbox{ si: } t_v< maxT \\
1 \mbox{ sinon }
\end{array}
\right.
$$ 

$$b=\sum_{i \in preds[v], t_i<t_v} \log \beta_{i,v}$$

Si $v$ appartient aux sources, on retourne $(a,b)=(1,0)$

Compléter la méthode $computeab$ donnée ci-dessous:   



In [7]:
eps=1e-20
from math import exp,log

def computeab(v, times, preds, maxT, eps=1e-20):
    preds = preds.get(v,[])
    t = times[v]
    if t==0:
        return (1,0)

    if len(preds)>0:
        # Tous les prédécesseurs dans c
        # Tous les ki des prédécesseurs dans k
        # Tous les ri des prédécesseurs dans r
        c, k, r= map(np.array,zip(*preds)) # mise en forme des prédécesseurs
        
        # Initialisation des deux sommes et sommation en même temps pour éviter
        # de faire deux boucles identiques
        somme_a = 0
        somme_b = 0
        for i in range(len(preds)):
            if times[c[i]] < t:
                # Calcul des sommes dans la même boucle
                delta_t = t - times[c[i]]
                alpha = k[i] * r[i] * exp(-r[i]*delta_t)
                beta = k[i] * exp(-r[i]*delta_t) + 1 - k[i]
                somme_a += alpha/beta
                somme_b += log(beta)
        # Si t=maxT alors on n'utilise pas la somme_A
        if t == maxT:
            return (1, somme_b)
        else: # Sinon on renvoit le max entre somme_A et epsilon
            return max(eps,somme_a), somme_b

nbNodes=len(graph[0])
times=np.array([maxT]*nbNodes,dtype=float)
times[0]=0
times[1]=1
times[2]=4

print(computeab(0,times,preds, maxT, eps))
print(computeab(1,times,preds, maxT, eps))
print(computeab(2,times,preds, maxT, eps))
print(computeab(3,times,preds, maxT, eps))


(1, 0)
(0.17610107365772137, -0.17810126145719926)
(0.012293749653343879, -0.21077360840944234)
(1, -1.1230118785518794)


Vérification : 


```
(1, 0)
(0.17610107365772135, -0.17810126145719926)
(0.012293749653343877, -0.2107736084094422)
(1.0, -1.12301187855188)
```



La méthode $computell$ calcule la log-vraisemblance d'une diffusion (représentée par le tableau times), en appelant la méthode computeab sur l'ensemble des noeuds du réseau. Elle retourne un triplet (log-likelihood, sa, sb), avec $sa$ et $sb$ les tables des valeurs $a$ et $b$ pour tous les noeuds.   

In [8]:
def computell(times,preds, maxT, eps):
    n = len(times)
    sa, sb = np.zeros((n)), np.zeros((n))
    ll = 0.0
    for v in range(n):
        #calcul de a,b pour tous les v
        a,b = computeab(v, times, preds, maxT)
        sa[v], sb[v] = a,b
    #calcul de la log-vraisemblance
        ll += log(a) + b
    return ll, sa, sb

ll,sa,sb=computell(times,preds, maxT, eps)
print("ll=",ll)
print(times)
print("like_indiv=",np.exp(np.log(sa)+sb))


ll= -13.117139892397578
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
like_indiv= [1.         0.14737154 0.00995741 0.32529856 0.1        0.52489353
 0.8        1.         0.20059727 1.         0.5       ]


Vérification : 



```
ll= -13.117139892397578
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
like_indiv= [1.         0.14737154 0.00995741 0.32529856 0.1        0.52489353
 0.8        1.         0.20059727 1.         0.5       ]
```



Afin de préparer les mises à jour lors des affectations successives des variables du Gibbs Sampling, on propose de définir une méthode $removeV(v,times,succs,sa,sb)$ qui retire temporairement du réseau un noeud $v$, en passant son temps d'infection à -1 dans times et en retirant sa contribution aux valeurs a et b (contenues dans sa et sb) de tous ses successeurs $j$ tels que $t_j > t_v$ (y compris donc les non infectés qui sont à $t_j=maxT$). 

In [10]:
def removeV(v,times,succs,sa,sb):
    succs=succs.get(v,[])
    t=times[v]
    if t<0:
        return 
    times[v]=-1
    sa[v]=1.0
    sb[v]=0.0
    if len(succs)>0:
        c,k,r=map(np.array,zip(*succs))
        tp=times[c]
        which=(tp>t)

        tp=tp[which]
        dt=tp-t
        k=k[which]
        r=r[which]
        c=c[which]
        rt = -r*dt
        b1=k*np.exp(rt)
        b=b1+1.0-k

        a=r*b1
        a=a/b
        b=np.log(b)

        sa[c]=sa[c]-np.where(tp<maxT,a,0.0)
        sa[c]=np.where(sa[c]>eps,sa[c],eps)
        sb[c]=sb[c]-b
        sb[c]=np.where(sb[c]>0,0,sb[c])


#Test
print("sa=",sa)
print("sb=",sb)

nsa=np.copy(sa)
nsb=np.copy(sb)
ntimes=np.copy(times)
removeV(3,ntimes,succs,nsa,nsb)
print("diffa=",nsa-sa)
print("diffb=",nsb-sb)

nsa=np.copy(sa)
nsb=np.copy(sb)
ntimes=np.copy(times)
removeV(1,ntimes,succs,nsa,nsb)
print("diffa=",nsa-sa)
print("diffb=",nsb-sb)


sa= [1.         0.17610107 0.01229375 1.         1.         1.
 1.         1.         1.         1.         1.        ]
sb= [ 0.         -0.17810126 -0.21077361 -1.12301188 -2.30258509 -0.64455983
 -0.22314355  0.         -1.60645602  0.         -0.69314718]
diffa= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
diffb= [0.         0.         0.         1.12301188 0.         0.
 0.         0.         0.         0.         0.        ]
diffa= [ 0.          0.82389893 -0.01229375  0.          0.          0.
  0.          0.          0.          0.          0.        ]
diffb= [0.         0.17810126 0.21077361 0.69314717 0.         0.
 0.22314355 0.         1.60645602 0.         0.69314718]


La méthode addVatT fait l'inverse: elle rajoute un noeud qui était retiré du réseau, avec un temps $newt$. Il faut alors mettre à jour les valeurs a et b (dans sa et sb) de tous les successeurs de $v$ tels que $t_j > newt$ et calculer les valeurs a et b du noeud v. 

Compléter le code ci-dessous: 

In [14]:
def addVatT(v,times,newt,preds,succs,sa,sb, maxT):
    t=times[v]
    if t>=0:
        raise Error("v  must have been removed before")
        
    #mise à jour du nouveau temps dans times
    times[v] = newt
    
    #mise à jour a et b dans les vecteurs sa et sb
    sa[v], sb[v] = computeab(v, times, preds, maxT, eps=1e-20)
    
    #mise à jour des tous les successeurs de v
    for j in preds:
        sa[j], sb[j] = computeab(j,times,preds,maxT)
    pass
        
# Tests: 
   
nsa=np.copy(sa)
nsb=np.copy(sb)
c,_,_=map(np.array,zip(*succs[1]))
c=np.append(c,1)
ll=np.sum((np.log(nsa)+nsb)[c])           # somme des logvraisemblances pouvant être modifiées par la modification du temps de 1 (avant modif)
removeV(1,times,succs,nsa,nsb)
addVatT(1,times,2,preds,succs,nsa,nsb, maxT)
ll2=np.sum((np.log(nsa)+nsb)[c])          # somme des logvraisemblances pouvant avoir été modifiées par la modification du temps de 1 (après modif)
removeV(1,times,succs,nsa,nsb)
addVatT(1,times,1,preds,succs,nsa,nsb, maxT)
ll3=np.sum((np.log(nsa)+nsb)[c])          # somme des logvraisemblances pouvant  avoir été modifiées par la modification du temps de 1 (après remise dans l'état initial)
llall=np.sum(np.log(nsa)+nsb)             # logvraisemblance globale
print(np.exp(ll),np.exp(ll2),np.exp(ll3),llall)

c,_,_=map(np.array,zip(*succs[0]))
c=np.append(c,0)
ll=np.sum((np.log(nsa)+nsb)[c])
removeV(0,times,succs,nsa,nsb)
addVatT(0,times,maxT,preds,succs,nsa,nsb, maxT)
ll2=np.sum((np.log(nsa)+nsb)[c])
removeV(0,times,succs,nsa,nsb)
addVatT(0,times,0,preds,succs,nsa,nsb, maxT)
ll3=np.sum((np.log(nsa)+nsb)[c])
llall=np.sum(np.log(nsa)+nsb)
print(np.exp(ll),np.exp(ll2),np.exp(ll3),llall)

c,_,_=map(np.array,zip(*succs[5]))
c=np.append(c,5)
ll=np.sum((np.log(nsa)+nsb)[c])
removeV(5,times,succs,nsa,nsb)
addVatT(5,times,1,preds,succs,nsa,nsb, maxT)
ll2=np.sum((np.log(nsa)+nsb)[c])
removeV(5,times,succs,nsa,nsb)
addVatT(5,times,maxT,preds,succs,nsa,nsb, maxT)
ll3=np.sum((np.log(nsa)+nsb)[c])
llall=np.sum(np.log(nsa)+nsb)
print(np.exp(ll),np.exp(ll2),np.exp(ll3),llall)


3.830251606174211e-05 8.555487921315824e-05 3.830251606174211e-05 -13.117139892397578
0.14737153555403676 1.0000000000169125e-21 0.14737153555403676 -13.117139892397578
0.524893534183932 2.999999999999998e-21 0.524893534183932 -13.117139892397578


Vérification : 



```
3.830251606174211e-05 8.555487921315824e-05 3.830251606174211e-05 -13.117139892397578
0.14737153555403676 1.0000000000169125e-21 0.14737153555403676 -13.117139892397578
0.5248935341839319 2.999999999999998e-21 0.5248935341839319 -13.117139892397578
```



Pour échantillonner pour une variable $i$, il faudra être à même de comparer les vraisemblances selon les différentes affectations. Cela implique de calculer la somme de toutes ces vraisemblances. Mais pour réaliser cette somme, il faudrait que nous sortions de la représentation logarithmique: $\sum_{t_i} exp(log(p(t_1,\dots,t_i,\dots,t_n))$. Si on le fait de cette manière, on risque d'avoir des arrondis à 0 presque partout. Une possibilité (log-sum-exp trick) est d'exploiter la relation suivante:  

$$\log\sum_i x_i = x^* + \log\left( \exp(x_1-x^*)+ \cdots + \exp(x_n-x^*) \right)$$
avec $x^* = \max{\{x_1, \dots, x_n\}}$

Compléter la méthode logsumexp suivante, qui réalise cette somme en évitant les problèmes numériques: 


In [15]:
def logsumexp(x,axis=-1):
    xe = np.max(x)
    return xe + np.log(np.sum(np.exp(x - xe),axis=1)) #somme sur les lignes
  

#Test: 
x=np.array([[0.001,0.02,0.008],[0.1,0.01,0.4]])
r=np.log(np.sum(x,-1))
x=np.log(x)
r2=logsumexp(x)
print(r2,r)

[-3.54045945 -0.67334455] [-3.54045945 -0.67334455]


Vérification : 


```
[-3.54045945 -0.67334455] [-3.54045945 -0.67334455]
```



On souhaite maintenant mettre en place une méthode $sampleV(v,times,newt,preds,succs,sa,sb,k,k2)$ qui sample un nouveau temps d'infection pour le noeud $v$, connaissant les temps de tous les autres noeuds dans $times$ (ainsi que leurs valeurs $a$ et $b$ correspondantes contenues dans sa et sb). Puisque le domaine de support de $t_v$ est continu, on doit faire quelques approximations en se basant sur une discrétisation des valeurs possibles:

1.   On découpe la plage de temps $[0;maxT]$ en $k$ bins réguliers. Dans chaque bin $i$, on échantillonne uniformément un temps, pour obtenir $k$ points $d_1,\dots,d_k$. Si $t_v < maxT$, on ajoute $t_v$ à cet ensemble de points pour gagner en stabilité (inséré dans la liste de manière à conserver l'ordre croissant). 
2.   On considère chaque point $d_i$ comme le prototype d'un bin $[(d_i+d_{i-1})/2,(d_i+d_{i+1})/2]$. Pour $d_1$ on prend $[0,(d_1+d_2)/2]$ et pour $d_k$ on prend   $[(d_k+d_{k-1})/2,maxT]$. On fait l'hypothèse que la densité de probabilité est constante sur l'ensemble de chaque bin $i$, que l'on évalue en  $t_v=d_i$.   La probabilité que l'on échantillonne dans le bin $i$ est alors égale à: $p(t_v \in bin_i | \{t_u\}_{ u \in V\setminus v}) =  \frac{z_i \times l_i}{\sum_j z_j \times l_j + z_{maxT}}$, avec $z_i$ la vraisemblance  calculée selon $t_v =d_i$, $l_i$ la taille du bin $i$ et $z_{maxT}$ la vraisemblance  calculée pour $t_v=maxT$. La probabilité que $v$ ne soit pas infecté dans la diffusion est alors donnée par : $p(t_v = maxT | \{t_u\}_{ u \in V\setminus v}) =  \frac{z_{maxT}}{\sum_j z_j \times l_j + z_{maxT}}$.
3. On échantillonne une variable $x$ proportionnellement aux probabilités calculées à l'étape précédente.  Si $x$ ne correspond pas à $maxT$, $v$ est alors infecté à un temps inclus dans l'intervale du bin correspondant à $x$. Il s'agit alors de re-échantillonner $k2$ points uniformément dans ce bin et de calculer les densités en ces points (pour gagner en stabilité on ajoute le prototype du bin $d_i$). Le nouveau temps de $v$ est alors échantillonné proportionnellement à ces densités.

Le code de la méthode de sampling est donné ci-dessous:





In [24]:
np.random.seed(0)



def getLL(v,times,nt,preds,succs,sa,sb, maxT, onUsers=None):
    sa=np.copy(sa)
    sb=np.copy(sb)
    if onUsers is None:
        onUsers=range(len(times))
        addVatT(v,times,nt,preds,succs,sa,sb, maxT)
        times[v]=-1
        ll=np.sum((np.log(sa)+sb)[onUsers])
    return (ll,sa,sb)

  
def sampleV(v,times,preds,succs,sa,sb,k,k2, maxT):
  
    nbCandidateT=k
    bounds=np.linspace(0,maxT,nbCandidateT)
    newt=np.random.uniform(bounds[:-1],bounds[1:])
  
    if times[v]<maxT:
        idx = newt.searchsorted(times[v])
        newt=np.concatenate((newt[:idx], [times[v]], newt[idx:]),axis=0)
        nbCandidateT+=1
    newt=np.append(newt,[maxT])
  
    if v in succs:
        c,_,_=map(list,zip(*succs.get(v,[])))
    else:
        c=[]
    c.append(v)
    c=np.array(c)
    oldll=np.sum((np.log(sa)+sb)[c])
    otime=times[v]
    nsa=np.copy(sa)
    nsb=np.copy(sb)
    removeV(v,times,succs,nsa,nsb)
    lls=[getLL(v,times,nt,preds,succs,nsa,nsb, maxT,onUsers=c) for nt in newt]
    ll,la,lb=zip(*lls)
    ll=list(ll)
    ll=np.array(ll)

    diffsx=(newt[1:]-newt[:-1])/2.0
    diffsx[1:]=diffsx[1:]+diffsx[:-1]
    diffsx[0]+=newt[0]
    diffsx[-1]+=(maxT-newt[nbCandidateT-1])/2.0
    areas=np.log(diffsx)+ll[:-1]
    lln=np.append(areas,ll[-1])
  

    p=np.exp(lln-logsumexp(lln))


    i=np.random.choice(range(len(p)),1,p=p).sum()
    if i==(len(p)-1):
        times[v]=maxT
        np.copyto(sa,np.array(la[-1]))
        np.copyto(sb,np.array(lb[-1]))
    else: 
        if i>0: 
            bi=(newt[i]+newt[i-1])/2.0
        else:
            bi=0
        if i<(len(p)-2): 
            bs=(newt[i]+newt[i+1])/2.0
        else:
            bs=maxT
            bounds=np.linspace(bi,bs,k2)
            newt=np.concatenate(([newt[i]],np.random.uniform(bounds[:-1],bounds[1:])))
            lls=[getLL(v,times,nt,preds,succs,nsa,nsb, maxT, onUsers=c) for nt in newt]
            ll,la,lb=zip(*lls)
            ll=np.array(ll)
            p=np.exp(ll-logsumexp(ll))

        i=np.random.choice(range(len(p)),1,p=p).sum()

        times[v]=newt[i]
        np.copyto(sa,np.array(la[i]))
        np.copyto(sb,np.array(lb[i]))

times=np.array([maxT]*nbNodes,dtype=float)
times[0]=0
times[1]=1
times[2]=4
np.random.seed(0)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)
sampleV(5,times,preds,succs,sa,sb,10,10, maxT)
print(times)


[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]


UnboundLocalError: local variable 'll' referenced before assignment

Vérification : 



```
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
[ 0.          1.          4.         10.         10.          4.20931617
 10.         10.         10.         10.         10.        ]
[ 0.  1.  4. 10. 10. 10. 10. 10. 10. 10. 10.]
```



Compléter la méthode de Gibbs Sampling $gb$ ci-dessous, avec $k$ le nombre de bins à utiliser et $k2$ le nombre de points à échantillonner dans le bin choisi. Le paramètre $ref$ correspond à un vecteur de probabilités marginales de référence (par exemple obtenu par MonteCarlo lorsque c'est possible) avec lequel on peut afficher la distance MSE au fur et à mesure du processus.

In [28]:
np.random.seed(1)
def gb(graph,infections, maxT, times, burnin=1000,nbEpochs=10000,k=100,k2=50, ref=None):
    eps = 1e-5
    rate = []
    preds, succs = getPredsSuccs(graph)
    sample = times.copy()
    #On génère tous les noeuds du système
    nbNodes = len(graph[0])
    #nombre d'échantillons à réaliser
    for i in range(nbEpochs):
        nodes = np.arange(nbNodes)
        #Pour chaque noeud on va générer un échantillon
        for j in range(nbNodes):
            ll, sa, sb = computell(times, preds, maxT,eps)
            v = np.random.choice(nodes)
            #On enlève ensuite chaque dans nodes pour ne pas retomber sur le même
            np.delete(nodes,v)
            #On échantillone selon les paramètres
            sampleV(v,times,preds,succs,sa,sb,k,k2)
            #Ensuite quand on a réalisé tous les échantillons on calcule la distance  mse = (x1 - x2)²
            if i > burnin:
                rate.append((times - ref)**2)
    return rate

# On teste ici avec seulement des sources (i.e., des infectés au temps 0), car cela permet de comparer à la ref MonteCarlo (mais il faudrait aussi tester avec d'autres infectés : c'est l'objectif). 
ref=getProbaMC(graph,[0], maxT) 
rate=gb(graph,[(0,0)],maxT, times, burnin=100,ref=ref)
print(rate)

KeyboardInterrupt: 

# Partie optionnelle

L'algorithme de Metropolis-Hasting est une autre méthode de type MCMC qui utilise une distribution d'échantillonnage pour se déplacer dans l'espace des points considérés. Il s'agit de définir une distribution $q(y_{t+1}|x_t)$ de laquelle on sait générer un déplacement. L'algorithme procéde alors de la manière suivante: 


1.   Générer $y_{t+1}$ selon $q(y_{t+1}|x_t)$ 
2.   Calculer la probabilité d’acceptation $\alpha(x_t,y_{t+1})=\min\left\{\frac{\pi(y_{t+1})q(x_t|y_{t+1})}{\pi(x_t)q(y_{t+1}|x_t)},1\right\} \,\!, \text{ avec } \pi(x_t) \text{ la densité de probabilité de } x_t$
3.   Prendre $x_{t+1}=\begin{cases} y_{t+1}, & \text{avec probabilité}\,\,\alpha \\ x_t, & \text{avec probabilité}\,\,1-\alpha \end{cases}$



Dans notre cas, on propose de travailler avec des déplacements correspondants à des permutations d'un temps d'infection à chaque itération, comme dans le cadre du Gibbs Sampling. A chaque étape on choisit donc une variable à modifier, on choisit un nouveau temps pour cette variable et on calcule la densité correspondante. La probabilité d'acceptation est ensuite calculée selon cette densité et la probabilité du déplacement selon la distribution $q$ qui a servi à générer le nouveau temps d'infection. On se propose de choisir $maxT$ avec une probabilité de 0.1. La probabilité $q(t_v|t)$ pour $t< maxT$ est alors égale  à  $0.9\times \frac{1}{maxT}$.

Implémenter l'approche d'échantillonnage par Metropolis-Hasting pour notre problème d'estimation de probabilités marginales d'infection. 

In [ ]:
# votre code ici

ref=getProbaMC(graph,[0]) 
rate=mh(graph,[(0,0)],burnin=100,ref=ref)
print(rate)